In [3]:
from collections import defaultdict, deque
import heapq
from typing import Dict, List, Set, Tuple, Optional

class Istasyon:
    def __init__(self, idx: str, ad: str, hat: str):
        self.idx = idx
        self.ad = ad
        self.hat = hat
        self.komsular: List[Tuple['Istasyon', int]] = []  # (istasyon, süre) tuple'ları

    def komsu_ekle(self, istasyon: 'Istasyon', sure: int):
        self.komsular.append((istasyon, sure))

class MetroAgi:
    def __init__(self):
        self.istasyonlar: Dict[str, Istasyon] = {}
        self.hatlar: Dict[str, List[Istasyon]] = defaultdict(list)

    def istasyon_ekle(self, idx: str, ad: str, hat: str) -> None:
        if idx not in self.istasyonlar:
            istasyon = Istasyon(idx, ad, hat)
            self.istasyonlar[idx] = istasyon
            self.hatlar[hat].append(istasyon)

    def baglanti_ekle(self, istasyon1_id: str, istasyon2_id: str, sure: int) -> None:
        istasyon1 = self.istasyonlar[istasyon1_id]
        istasyon2 = self.istasyonlar[istasyon2_id]
        istasyon1.komsu_ekle(istasyon2, sure)
        istasyon2.komsu_ekle(istasyon1, sure)
    
    def en_az_aktarma_bul(self, baslangic_id: str, hedef_id: str) -> Optional[List[Istasyon]]:
        if baslangic_id not in self.istasyonlar or hedef_id not in self.istasyonlar:
            return None
        
        baslangic = self.istasyonlar[baslangic_id]
        hedef = self.istasyonlar[hedef_id]
        
        kuyruk = deque([(baslangic, [baslangic])])
        ziyaret_edildi = set([baslangic])
        
        while kuyruk:
            current, rota = kuyruk.popleft()
            
            if current == hedef:
                return rota
            
            for komsu, _ in current.komsular:
                if komsu not in ziyaret_edildi:
                    ziyaret_edildi.add(komsu)
                    yeni_rota = rota + [komsu]
                    kuyruk.append((komsu, yeni_rota))
        
        return None

    def en_hizli_rota_bul(self, baslangic_id: str, hedef_id: str) -> Optional[Tuple[List[Istasyon], int]]:
        if baslangic_id not in self.istasyonlar or hedef_id not in self.istasyonlar:
            return None

        baslangic = self.istasyonlar[baslangic_id]
        hedef = self.istasyonlar[hedef_id]
        
        # Öncelik kuyruğu: (toplam tahmini süre, id(current), current, rota, toplam süre)
        pq = [(0, id(baslangic), baslangic, [baslangic], 0)]
        ziyaret_edildi = {}
        
        while pq:
            _, _, current, rota, toplam_sure = heapq.heappop(pq)
            
            if current == hedef:
                return (rota, toplam_sure)
            
            if current.idx in ziyaret_edildi and ziyaret_edildi[current.idx] <= toplam_sure:
                continue
                
            ziyaret_edildi[current.idx] = toplam_sure
            
            for komsu, sure in current.komsular:
                yeni_toplam_sure = toplam_sure + sure
                if komsu.idx not in ziyaret_edildi or yeni_toplam_sure < ziyaret_edildi[komsu.idx]:
                    yeni_rota = rota + [komsu]
                    heapq.heappush(pq, (yeni_toplam_sure, id(komsu), komsu, yeni_rota, yeni_toplam_sure))
        
        return None

# İstanbul Metrosu Örnek Kullanım
if __name__ == "__main__":
    metro = MetroAgi()
    
    # M1A Yenikapı-Atatürk Havalimanı Hattı
    metro.istasyon_ekle("M1A1", "Yenikapı", "M1A")
    metro.istasyon_ekle("M1A2", "Aksaray", "M1A")
    metro.istasyon_ekle("M1A3", "Emniyet", "M1A")
    metro.istasyon_ekle("M1A4", "Ulubatlı", "M1A")
    metro.istasyon_ekle("M1A5", "Atatürk Havalimanı", "M1A")
    
    # M2 Yenikapı-Hacıosman Hattı
    metro.istasyon_ekle("M2A1", "Yenikapı", "M2")
    metro.istasyon_ekle("M2A2", "Vezneciler", "M2")
    metro.istasyon_ekle("M2A3", "Taksim", "M2")
    metro.istasyon_ekle("M2A4", "Levent", "M2")
    metro.istasyon_ekle("M2A5", "Hacıosman", "M2")
    
    # M3 Kirazlı-Başakşehir Hattı
    metro.istasyon_ekle("M3A1", "Kirazlı", "M3")
    metro.istasyon_ekle("M3A2", "Olimpiyat", "M3")
    metro.istasyon_ekle("M3A3", "Başakşehir", "M3")
    
    # M4 Kadıköy-Tavşantepe Hattı
    metro.istasyon_ekle("M4A1", "Kadıköy", "M4")
    metro.istasyon_ekle("M4A2", "Ayrılık Çeşmesi", "M4")
    metro.istasyon_ekle("M4A3", "Ünalan", "M4")
    metro.istasyon_ekle("M4A4", "Tavşantepe", "M4")
    
    # Marmaray (Banliyö Treni)
    metro.istasyon_ekle("MR1", "Halkalı", "Marmaray")
    metro.istasyon_ekle("MR2", "Yenikapı", "Marmaray")
    metro.istasyon_ekle("MR3", "Ayrılık Çeşmesi", "Marmaray")
    metro.istasyon_ekle("MR4", "Gebze", "Marmaray")
    
    # Bağlantılar ekleme
    # M1A bağlantıları
    metro.baglanti_ekle("M1A1", "M1A2", 3)  # Yenikapı - Aksaray
    metro.baglanti_ekle("M1A2", "M1A3", 2)  # Aksaray - Emniyet
    metro.baglanti_ekle("M1A3", "M1A4", 4)  # Emniyet - Ulubatlı
    metro.baglanti_ekle("M1A4", "M1A5", 5)  # Ulubatlı - Atatürk Havalimanı
    
    # M2 bağlantıları
    metro.baglanti_ekle("M2A1", "M2A2", 4)  # Yenikapı - Vezneciler
    metro.baglanti_ekle("M2A2", "M2A3", 5)  # Vezneciler - Taksim
    metro.baglanti_ekle("M2A3", "M2A4", 4)  # Taksim - Levent
    metro.baglanti_ekle("M2A4", "M2A5", 3)  # Levent - Hacıosman
    
    # M3 bağlantıları
    metro.baglanti_ekle("M3A1", "M3A2", 6)  # Kirazlı - Olimpiyat
    metro.baglanti_ekle("M3A2", "M3A3", 5)  # Olimpiyat - Başakşehir
    
    # M4 bağlantıları
    metro.baglanti_ekle("M4A1", "M4A2", 3)  # Kadıköy - Ayrılık Çeşmesi
    metro.baglanti_ekle("M4A2", "M4A3", 2)  # Ayrılık Çeşmesi - Ünalan
    metro.baglanti_ekle("M4A3", "M4A4", 4)  # Ünalan - Tavşantepe
    
    # Marmaray bağlantıları
    metro.baglanti_ekle("MR1", "MR2", 10)  # Halkalı - Yenikapı
    metro.baglanti_ekle("MR2", "MR3", 8)   # Yenikapı - Ayrılık Çeşmesi
    metro.baglanti_ekle("MR3", "MR4", 30)  # Ayrılık Çeşmesi - Gebze
    
    # Aktarma bağlantıları
    metro.baglanti_ekle("M1A1", "M2A1", 2)  # Yenikapı aktarma (M1A-M2)
    metro.baglanti_ekle("M1A1", "MR2", 3)   # Yenikapı aktarma (M1A-Marmaray)
    metro.baglanti_ekle("M2A1", "MR2", 3)   # Yenikapı aktarma (M2-Marmaray)
    metro.baglanti_ekle("M4A2", "MR3", 2)   # Ayrılık Çeşmesi aktarma (M4-Marmaray)
    
    # Test senaryoları
    print("\n=== İstanbul Metrosu Test Senaryoları ===")
    
    # Senaryo 1: Kadıköy'den Taksim'e
    print("\n1. Kadıköy'den Taksim'e:")
    rota = metro.en_az_aktarma_bul("M4A1", "M2A3")
    if rota:
        print("En az aktarmalı rota:", " -> ".join(f"{i.ad} ({i.hat})" for i in rota))
    
    sonuc = metro.en_hizli_rota_bul("M4A1", "M2A3")
    if sonuc:
        rota, sure = sonuc
        print(f"En hızlı rota ({sure} dakika):", " -> ".join(f"{i.ad} ({i.hat})" for i in rota))
    
    # Senaryo 2: Atatürk Havalimanı'ndan Kadıköy'e
    print("\n2. Atatürk Havalimanı'ndan Kadıköy'e:")
    rota = metro.en_az_aktarma_bul("M1A5", "M4A1")
    if rota:
        print("En az aktarmalı rota:", " -> ".join(f"{i.ad} ({i.hat})" for i in rota))
    
    sonuc = metro.en_hizli_rota_bul("M1A5", "M4A1")
    if sonuc:
        rota, sure = sonuc
        print(f"En hızlı rota ({sure} dakika):", " -> ".join(f"{i.ad} ({i.hat})" for i in rota))
    
    # Senaryo 3: Halkalı'dan Gebze'ye
    print("\n3. Halkalı'dan Gebze'ye:")
    rota = metro.en_az_aktarma_bul("MR1", "MR4")
    if rota:
        print("En az aktarmalı rota:", " -> ".join(f"{i.ad} ({i.hat})" for i in rota))
    
    sonuc = metro.en_hizli_rota_bul("MR1", "MR4")
    if sonuc:
        rota, sure = sonuc
        print(f"En hızlı rota ({sure} dakika):", " -> ".join(f"{i.ad} ({i.hat})" for i in rota))
    
    # Senaryo 4: Başakşehir'den Levent'e
    print("\n4. Başakşehir'den Levent'e:")
    rota = metro.en_az_aktarma_bul("M3A3", "M2A4")
    if rota:
        print("En az aktarmalı rota:", " -> ".join(f"{i.ad} ({i.hat})" for i in rota))
    
    sonuc = metro.en_hizli_rota_bul("M3A3", "M2A4")
    if sonuc:
        rota, sure = sonuc
        print(f"En hızlı rota ({sure} dakika):", " -> ".join(f"{i.ad} ({i.hat})" for i in rota)) 


=== İstanbul Metrosu Test Senaryoları ===

1. Kadıköy'den Taksim'e:
En az aktarmalı rota: Kadıköy (M4) -> Ayrılık Çeşmesi (M4) -> Ayrılık Çeşmesi (Marmaray) -> Yenikapı (Marmaray) -> Yenikapı (M2) -> Vezneciler (M2) -> Taksim (M2)
En hızlı rota (25 dakika): Kadıköy (M4) -> Ayrılık Çeşmesi (M4) -> Ayrılık Çeşmesi (Marmaray) -> Yenikapı (Marmaray) -> Yenikapı (M2) -> Vezneciler (M2) -> Taksim (M2)

2. Atatürk Havalimanı'ndan Kadıköy'e:
En az aktarmalı rota: Atatürk Havalimanı (M1A) -> Ulubatlı (M1A) -> Emniyet (M1A) -> Aksaray (M1A) -> Yenikapı (M1A) -> Yenikapı (Marmaray) -> Ayrılık Çeşmesi (Marmaray) -> Ayrılık Çeşmesi (M4) -> Kadıköy (M4)
En hızlı rota (30 dakika): Atatürk Havalimanı (M1A) -> Ulubatlı (M1A) -> Emniyet (M1A) -> Aksaray (M1A) -> Yenikapı (M1A) -> Yenikapı (Marmaray) -> Ayrılık Çeşmesi (Marmaray) -> Ayrılık Çeşmesi (M4) -> Kadıköy (M4)

3. Halkalı'dan Gebze'ye:
En az aktarmalı rota: Halkalı (Marmaray) -> Yenikapı (Marmaray) -> Ayrılık Çeşmesi (Marmaray) -> Gebze (Marmar